# Sending SQuaSH data to InfluxDB

In this notebook we reproduce what the InfluxDB Cellery task in the SQuaSH API does, and execute this code to synchronize the SQuaSH production data with an InfluxDB instance. See also [this notebook](https://github.com/lsst-sqre/influx-demo) for a quick introduction on InfluxDB concepts.

In [ ]:
SQUASH_API_URL = "https://squash-restful-api.lsst.codes/"
INFLUXDB_API_URL = "https://influxdb-demo.lsst.codes"

We start by creating a new database. Note that if the database already exists nothing is done (the existing data is preserved), and an status code 200 (OK) is returned.

In [ ]:
import requests
import json

INFLUXDB_DATABASE = "squash-prod"

params={'q': 'CREATE DATABASE "{}"'.format(INFLUXDB_DATABASE)}
r = requests.post(url=INFLUXDB_API_URL + "/query", params=params)
r.status_code

The following cells will grab SQuaSH data and write it in the format used by InfluxDB (called [line protocol](https://docs.influxdata.com/influxdb/v1.6/write_protocols/line_protocol_tutorial/)):


```#<measurement>[,<tag_key>=<tag_value>[,<tag_key>=<tag_value>]] <field_key>=<field_value>[,<field_key>=<field_value>] [<timestamp>]```

the important thing here is that an InfluxDB measurement is equivalent to a "table", tags are annotations that are used to query the data, and thus are indexed. Fields correspond to the actual values, and the timestamp acts like the "primary key" in InfluxDB.

As you run this notebook you might follow the data ingestion using the [Data Explorer](https://chronograf-demo.lsst.codes/) tool in Chronograf.

In [ ]:
import os
import math
from pytz import UTC
from datetime import datetime
from dateutil.parser import parse

In [ ]:
def format_timestamp(date):
    """ Format timestamp as required by the InfluxDB line protocol.

        Parameters
        ----------
        date: `<str>`
            Timestamp string

        Returns
        -------
        timestamp: `<int>`
            Timestamp in nanosecond-precision Unix time.
            See https://docs.influxdata.com/influxdb/v1.6/write_protocols/
    """

    epoch = UTC.localize(datetime.utcfromtimestamp(0))

    timestamp = int((parse(date) - epoch).total_seconds() * 1e9)

    return timestamp


In [ ]:
def format_line(measurement, tags, fields, timestamp):
    """ Format an InfluxDB line.

        Parameters
        ----------
        measurement: `<str>`
            Name of the InfluxDB measurement
        tags: `<list>`
            A list of valid InfluxDB tags
        fields: `<list>`
            A list of valid InfluxDB fields
        timestamp: `int`
            A timestamp as returned by `format_timestamp()`

        Returns
        -------
        line: `<str>`
            An InfluxDB line as defined by the line protocol in
            https://docs.influxdata.com/influxdb/v1.6/write_protocols/
    """
    line = "{},{} {} {}".format(measurement, ",".join(tags), ",".join(fields),
                                timestamp)
    return line


In [ ]:
def send_to_influxdb(influxdb_line):
    """ Send a line to an InfluxDB database. It assumes the database already
        exists.

        Parameters
        ----------
        influxdb_line: `<str>`
            An InfluxDB line as defined by the line protocol in
            https://docs.influxdata.com/influxdb/v1.6/write_protocols/

        status_code: `<int>`
            Status code from the InfluxDB HTTP API.
    """

    params = {'db': INFLUXDB_DATABASE}
    r = requests.post(url=INFLUXDB_API_URL + "/write", params=params,
                      data=influxdb_line)

    return r.status_code, r.text



In [ ]:

def job_to_influxdb(job_id, data):
    """ Unpack a SQuaSH job and send to InfluxDB

        Parameters
        ----------
        data: `<dict>`
            A dictionary containing the job data

        Returns
        -------
        status_code: `<int>`
             204:
               The request was processed successfully
             400:
               Malformed syntax or bad query

        Note
        ----
        A lsst.verify measurement and an InfluxDB measurement are different
        things. See e.g.:
        https://docs.influxdata.com/influxdb/v1.6/concepts/key_concepts/
    """

    # The datamodel for a SQuaSH job in InfluxDB maps each verification package
    # to an InfluxDB measurement, verification job metadata to InfluxDB
    # tags and metric names and values to fields.

    # Here we associate metrics (fields) to their corresponding
    # packages (measurements)

    fields = {}
    for meas in data['measurements']:
        influxdb_measurement = meas['metric'].split('.')[0]

        if influxdb_measurement not in fields:
            fields[influxdb_measurement] = []

        if not math.isnan(meas['value']):
            fields[influxdb_measurement].append("{}={}".format(meas['metric'],
                                                               meas['value']))

    timestamp = format_timestamp(data['date_created'])

    # add the squash job_id and the ci_dataset as metadata
    data['meta']['squash_job_id'] = job_id

    if 'ci_dataset' in data['meta']['env']:
        data['meta']['ci_dataset'] = data['meta']['env']['ci_dataset']

    # skip other job metadata when forming tags
    del data['meta']['env']
    del data['meta']['packages']

    tags = []
    for key, value in data['meta'].items():
        # tag values cannot have blank spaces
        if type(value) == str:
            value = value.replace(" ", "_")
            tags.append("{}={}".format(key, value))
        else:
            tags.append("{}={}".format(key, value))

    for measurement in fields:
        influxdb_line = format_line(measurement, tags, fields[measurement],
                                    timestamp)

        status_code, message = send_to_influxdb(influxdb_line)
        if status_code != 204:
            print(message)

    return 

In [ ]:
jobs = requests.get(SQUASH_API_URL + "/jobs").json()

In [ ]:
for job_id in jobs['ids']:

    data = requests.get(SQUASH_API_URL + "/job/{}".format(job_id)).json()

    # Skip datasets we don't want 
    if data['ci_dataset'] == 'unknown' or data['ci_dataset'] == 'decam':
        continue

    print('Sending line for job {}...'.format(job_id))
    
    job_to_influxdb(job_id, data)
    